<a href="https://colab.research.google.com/github/eldercamposds/RegressaoPySpark/blob/main/RegresaoPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


In [3]:
from pyspark.sql import SparkSession

# Criand uma sessão Spark
spark = SparkSession.builder \
    .appName("Regressao") \
    .getOrCreate()

# Verificando sessão Spark
print(spark.version)


3.5.5


# Linear Regression

In [4]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
from pyspark.ml.feature import VectorAssembler #oneHotEcoder

In [11]:
carros =  spark.read.csv("/content/drive/MyDrive/Colab Notebooks/PySpark/Spark/download/Carros.csv", header=True, inferSchema=True, sep=";")

In [12]:
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [13]:
carros_2 = carros.select("Consumo", "Cilindros", "Cilindradas", "HP")

In [14]:
carros_2.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



In [17]:
vec_caracteristicas = VectorAssembler(inputCols=["Consumo", "Cilindros", "Cilindradas"], outputCol="caracteristicas")

In [18]:
carros_2 = vec_caracteristicas.transform(carros_2)

In [19]:
carros_2.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



In [25]:
carros_2.count()

32

In [20]:
treino, teste = carros_2.randomSplit([0.7, 0.3])

In [23]:
treino.count()

24

In [24]:
teste.count()

8

In [26]:
reglin =  LinearRegression(featuresCol="caracteristicas", labelCol="HP")

In [27]:
modelo = reglin.fit(treino)

In [28]:
previsao = modelo.transform(teste)

In [29]:
previsao.show(5)

+-------+---------+-----------+---+-----------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     26|        4|       1203| 91|[26.0,4.0,1203.0]| 104.4478729893712|
|    104|        8|        472|205|[104.0,8.0,472.0]|197.57734027087264|
|    133|        8|        350|245|[133.0,8.0,350.0]|193.57366951835417|
|    147|        8|        440|230|[147.0,8.0,440.0]| 191.4651421908542|
|    158|        8|        351|264|[158.0,8.0,351.0]| 189.9969294896363|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



In [30]:
avaliar = RegressionEvaluator(predictionCol="prediction", labelCol="HP", metricName="rmse")

In [31]:
rmse = avaliar.evaluate(previsao)

In [34]:
print(rmse)

39.720096646998215


# Radon Forest Regressor



In [37]:
rfreg = RandomForestRegressor(featuresCol="caracteristicas", labelCol="HP")

In [38]:
modelo2 = rfreg.fit(treino)

In [41]:
previsao2 = modelo2.transform(teste)

In [42]:
previsao2.show(5)

+-------+---------+-----------+---+-----------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|        prediction|
+-------+---------+-----------+---+-----------------+------------------+
|     26|        4|       1203| 91|[26.0,4.0,1203.0]| 132.3061111111111|
|    104|        8|        472|205|[104.0,8.0,472.0]| 207.7607142857143|
|    133|        8|        350|245|[133.0,8.0,350.0]| 212.5154761904762|
|    147|        8|        440|230|[147.0,8.0,440.0]|203.33214285714286|
|    158|        8|        351|264|[158.0,8.0,351.0]| 186.4529761904762|
+-------+---------+-----------+---+-----------------+------------------+
only showing top 5 rows



In [43]:
rmse2 = avaliar.evaluate(previsao2)

In [44]:
print(rmse2)

42.923962070054976


In [45]:
print(rmse)

39.720096646998215
